In [1]:
#You don't need to change anything in this block, although the modules need to be installed to run this notebook

#We import numpy to handle vectors and some math
import numpy as np

#We import pandas to create a data frame of the experiment data
#Such a table can later be used for plotting our results
import pandas as pd

# Import plotly, which is used for visualization
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

In [2]:
# generate N random particles in a 2d environment with a given pattern:
def generate_pattern(N, dim=2):
    return np.random.rand(N, dim)*2.0 - 1.0

In [3]:
# record data into dataframe (used for plotting)
def make_df(data, t, type_name, **kwargs):
    df = pd.DataFrame(data, columns=[f"x{i}" for i,_ in enumerate(data[0])])
    df['t'] = t
    df['type'] = type_name
    df['pid'] = range(len(data))
    for k, v in kwargs.items():
        df[k] = v
    return df

In [4]:
from pymoo import problems
dim = 2
ackley = problems.get_problem("rastrigin", n_var=dim)

def f2(x):
    return 5 * ackley.evaluate(x)[0]

def f1(x):
    return np.linalg.norm(x)

In [5]:
# force for particle i
def random_phi(dim):
    return np.random.rand(dim)
    #return np.random.rand()

def force_i(i, particles, x_l, x_p, c1=2.0, c2=2.0):
    return c1 * random_phi(dim) * (x_l - particles[i]) + c2 * random_phi(dim) * (x_p[i] - particles[i])
    
# forces for all particles
def force(particles, x_l, x_p, c1=2.0, c2=2.0):
    return np.array([force_i(i, particles, x_l, x_p, c1=c1, c2=c2) for i, _ in enumerate(particles)])


In [6]:
# run an experiment with N random particles and a random pattern
l = []
data = []
N = 50

w = 0.4
c1 = 2.0
c2 = 2.0

def fitness(x):
    return np.array([f2(x_i) for x_i in x])

# use 31 runs, when testing more which type runs better
for run in range(31):
    x = generate_pattern(N, dim=dim)
    v = np.zeros_like(x)
    # fitness of population
    f = fitness(x)
    # fitness of previous best
    f_p = fitness(x)
    # fitness of local best (global best for fully connected)
    f_l = min(fitness(x))
    x_l = x[np.argmin(fitness(x))]
    x_p = x
    
    for t in range(100):
        # record the current state at time t
        data.append(make_df(x.copy(), t, "particle", fitness=f, best_fitness=f_l, previous_bestf=f_p, run=run))
        # update v and x
        v = force(x, x_l, x_p, c1=c1, c2=c2) + w * v
        x = x + v
        # compute fitness
        f = fitness(x)
        # update x_p
        x_p[f < f_p] = x[f < f_p]
        # update f_p
        f_p[f < f_p] = f[f < f_p]
        # update x_l
        if f_l > min(f):
            x_l = x[np.argmin(f)]
            f_l = min(f)
df = pd.concat(data)


In [7]:
fig = px.line(df, x='t', y='best_fitness', color='run')
fig.update_layout(yaxis_range=(0, None))
fig.show()

In [8]:
fig = px.box(df, x='t', y='best_fitness')
fig.update_layout(yaxis_range=(0, None))
fig.show()

#Solution

#Part A

In [9]:
# generate N random particles in a 2d environment with a given pattern:
def generate_pattern(N, dim=2):
    return np.random.rand(N, dim)*0.5 + 0.5

# run an experiment with N random particles and a random pattern
l = []
data = []
N = 50

w = 0.4
c1 = 2.0
c2 = 2.0

def fitness(x):
    return np.array([f2(x_i) for x_i in x])

# use 31 runs, when testing more which type runs better
for run in range(31):
    x = generate_pattern(N, dim=dim)
    v = np.zeros_like(x)
    # fitness of population
    f = fitness(x)
    # fitness of previous best
    f_p = fitness(x)
    # fitness of local best (global best for fully connected)
    f_l = min(fitness(x))
    x_l = x[np.argmin(fitness(x))]
    x_p = x

    for t in range(100):
        # record the current state at time t
        data.append(make_df(x.copy(), t, "particle", fitness=f, best_fitness=f_l, previous_bestf=f_p, run=run))
        # update v and x
        v = force(x, x_l, x_p, c1=c1, c2=c2) + w * v
        x = x + v
        # compute fitness
        f = fitness(x)
        # update x_p
        x_p[f < f_p] = x[f < f_p]
        # update f_p
        f_p[f < f_p] = f[f < f_p]
        # update x_l
        if f_l > min(f):
            x_l = x[np.argmin(f)]
            f_l = min(f)

df = pd.concat(data)

In [10]:
# show the result
fig = px.scatter(df.loc[df.run.eq(0)], x="x0", y="x1", color="pid", animation_frame="t", animation_group="pid")
fig.update_layout(xaxis_range=(-1, 2), yaxis_range=(-1, 2), width=800, height=800)
fig.update_traces(marker={"size": 12})
fig.show()

In [11]:
fig = px.line(df, x='t', y='best_fitness', color='run')
fig.update_layout(yaxis_range=(0, None))
fig.show()

In [12]:
fig = px.box(df, x='t', y='best_fitness')
fig.update_layout(yaxis_range=(0, None))
fig.show()

#Part B

In [13]:
# generate N random particles in a 2d environment with a given pattern:
def generate_pattern(N, dim=2):
    return np.random.rand(N, dim)*0.5 + 0.5

# run an experiment with N random particles and a random pattern
l = []
data = []
N = 50

w = 0.4
c1 = 2.0
c2 = 2.0

# Assign charges to the first N/5 particles (i.e., 10)
charges = [0 for _ in range(N)]
for idx in range(int(N/5)):
    charges[idx] = np.random.rand()

def coulomb(i_pos, j_pos, i_charge, j_charge, k=2.0, dist_thresh=0.8):
    if np.linalg.norm(i_pos - j_pos) < dist_thresh:
        return k * (i_charge*j_charge*(i_pos-j_pos))/(np.linalg.norm(i_pos - j_pos)**3)
    else:
        return 0

def charged_force_sum_i(i, particles, charges):
    sum = 0
    for idx in range(len(particles)):
        if i != idx:
            sum += coulomb(particles[i], particles[idx], charges[i], charges[idx])
        else:
            sum += 0
    return sum

def force_i(i, particles, x_l, x_p, charges, c1=2.0, c2=2.0):
    return c1 * random_phi(dim) * (x_l - particles[i]) + c2 * random_phi(dim) * (x_p[i] - particles[i]) + charged_force_sum_i(i, particles, charges)

# forces for all particles
def force(particles, x_l, x_p, charges, c1=2.0, c2=2.0):
    return np.array([force_i(i, particles, x_l, x_p, charges, c1=c1, c2=c2) for i, _ in enumerate(particles)])

def fitness(x):
    return np.array([f2(x_i) for x_i in x])

# use 31 runs, when testing more which type runs better
for run in range(31):
    x = generate_pattern(N, dim=dim)
    v = np.zeros_like(x)
    # fitness of population
    f = fitness(x)
    # fitness of previous best
    f_p = fitness(x)
    # fitness of local best (global best for fully connected)
    f_l = min(fitness(x))
    x_l = x[np.argmin(fitness(x))]
    x_p = x

    for t in range(100):
        # record the current state at time t
        data.append(make_df(x.copy(), t, "particle", fitness=f, best_fitness=f_l, previous_bestf=f_p, run=run))
        # update v and x
        v = force(x, x_l, x_p, charges, c1=c1, c2=c2) + w * v
        x = x + v
        # compute fitness
        f = fitness(x)
        # update x_p
        x_p[f < f_p] = x[f < f_p]
        # update f_p
        f_p[f < f_p] = f[f < f_p]
        # update x_l
        if f_l > min(f):
            x_l = x[np.argmin(f)]
            f_l = min(f)

df = pd.concat(data)

In [14]:
# show the result
fig = px.scatter(df.loc[df.run.eq(0)], x="x0", y="x1", color="pid", animation_frame="t", animation_group="pid")
fig.update_layout(xaxis_range=(-1, 2), yaxis_range=(-1, 2), width=800, height=800)
fig.update_traces(marker={"size": 12})
fig.show()

In [15]:
fig = px.line(df, x='t', y='best_fitness', color='run')
fig.update_layout(yaxis_range=(0, None))
fig.show()

In [16]:
fig = px.box(df, x='t', y='best_fitness')
fig.update_layout(yaxis_range=(0, None))
fig.show()